In [2]:
import os

from dotenv import load_dotenv
from google.genai import Client, types
from pydantic import BaseModel, Field
from rich import print

load_dotenv()


class Chapter(BaseModel):
    header: str = Field(description="A descriptive title for the chapter")
    key_points: list[str] = Field(description="Important takeaways and insights from this chapter")
    summary: str = Field(description="A comprehensive summary of the chapter content")


class Analysis(BaseModel):
    title: str = Field(description="The main title or topic of the video content")
    chapters: list[Chapter] = Field(description="Structured breakdown of content into logical chapters")
    key_facts: list[str] = Field(description="Important facts, statistics, or data points mentioned")
    takeaways: list[str] = Field(description="Key insights and actionable takeaways for the audience")
    overall_summary: str = Field(description="A comprehensive summary synthesizing all chapters, facts, and themes")


client = Client(
    api_key=os.getenv("GEMINI_API_KEY"),
    http_options={"timeout": 600000},
)


response = client.models.generate_content_stream(
    model="models/gemini-2.5-pro",
    contents=types.Content(
        parts=[
            types.Part(file_data=types.FileData(file_uri="https://youtu.be/0kJKfcfE-Ck")),
        ]
    ),
    config=types.GenerateContentConfig(
        system_instruction="Analyze the video/transcript according to the schema and follow the original language.",
        temperature=0,
        response_mime_type="application/json",
        response_schema=Analysis,
        thinking_config=types.ThinkingConfig(thinking_budget=2048),
    ),
)


result_parts = [chunk.text for chunk in response if chunk.text is not None and (print(chunk.text, end="") or True)]
final_result = "".join(result_parts)

final_result = Analysis.model_validate_json(final_result)

print(final_result)

{
  "title": "美股24小時交易：如何

操作、優缺點及應用策略",
  "chapters": [
    {
      "header": "

什麼是美股24小時交易？",
      "key_points": [
        "美股可透過電子

通訊網絡（ECN）平台實現24小時交易。",
        "除了傳統交易所，還有ARCA、EDGX

、Blue Ocean等ECN平台提供延長交易時段。",
        "ECN平台與傳統交易所交易的

...

可以在白天交易美股。影片提供了在Interactive Brokers平台上的詳細操作演示，並深入分析了24小時交易的利

弊。其主要優點是便利性，但缺點是流通量低和交易成本可能增加（差價擴大

）。講者建議，投資者應理解真正的價格發現主要發生在正常交易時段，因此通宵交易更

適合用來執行預先設定好的目標價訂單，而不是用來判斷市場走勢。"
}

{
  "title": "美股24小時交易：如何操作、優缺點及應用策略",
  "chapters": [
    {
      "header": "什麼是美股24小時交易？",
      "key_points": [
        "美股可透過電子通訊網絡（ECN）平台實現24小時交易。",
        "除了傳統交易所，還有ARCA、EDGX、Blue Ocean等ECN平台提供延長交易時段。",
        "ECN平台與傳統交易所交易的股票完全相同，只是交易渠道不同。"
      ],
      "summary": 
"影片介紹了美股現在可以24小時交易，解決了亞洲投資者需要熬夜的問題。這主要是通過傳統交易所（如紐約證券交易所、納斯達
克）之外的電子通訊網絡（ECN）平台實現的。這些ECN平台，如ARCA、EDGX和Blue 
Ocean，提供了盤前、盤後甚至通宵的交易時段，讓交易時間得以延長。這些平台與傳統交易所提供的股票是完全相同的，只是交易
渠道和時間不同。"
    },
    {
      "header": "24小時美股交易實況操作",
      "key_points": [
        "使用IB平台時，需在「Time In Force (TIF)」中選擇「Overnight Trading」。",
        "影片以買入特斯拉股票為例進行操作示範。",
        "下單前務必覆核訂單總金額，防止出錯。",
        "通宵交易時段只接受限價單（Limit Order）。"
      ],
      "summary": "影片以Interactive Brokers (IB) 
的交易平台為例，逐步示範如何在香港的日間時段（對應美國的夜間）買賣美股。操作關鍵在於下單時，在「Time In Force 
(TIF)」選項中選擇「Overnight 
Trading」。影片展示了如何為特斯拉（TSLA）股票下達一個通宵交易的買單，並強調了檢查訂單總金額（Amount）的重要性，以避
免下錯單。此外，通宵交易目前只接受限價單（Limit Order）。"
    },
    {
      "header": "24小時交易的優點與缺點",
      "key_points": [
        "優點：方便，可在亞洲時區對市場新聞作出即時反應。",
        "缺點：流通量低，買賣差價闊。",
        "缺點：訂單稀疏，大額交易容易造成滑價（Trade Slippage）。"
      ],
      "summary": 
"24小時交易的最大好處是方便，尤其對於亞洲投資者，可以在白天對突發事件作出反應，無需熬夜。然而，缺點也很明顯，主要是
在非正常交易時段（特別是通宵時段），市場流通量低，導致掛盤稀疏、買賣差價擴大，大額交易容易影響價格並造成滑價（Slipp
age）風險。"
    },
    {
      "header": "如何應用24小時交易策略",
      "key_points": [
        "真正的價格發現（Price Discovery）發生在正常交易時段。",
        "不應根據通宵時段的價格波動來判斷市場趨勢或做交易決策。",
        "可利用通宵交易時段執行預先設定好的目標價位訂單。"
      ],
      "summary": "講者分享了他的個人策略。他認為真正的價格發現（Price 
Discovery）發生在交易量最大的正常交易時段（美國白天）。因此，他不會根據通宵時段的價格波動來做新的交易決策。相反，他
會利用這個時段來執行預先設定好的目標價位訂單，例如當股價達到某個預設的賣出價時，就可以掛單賣出。他用街市買魚作比喻
，日間的價格才反映真實的供需，深夜的零星交易價格不具代表性。"
    }
  ],
  "key_facts": [
    "美股可透過ECN平台實現24小時交易。",
    "Blue Ocean ATS平台提供美國東岸時間晚上8點至凌晨4點的交易，對應香港夏令時間早上8點至下午4點。",
    "通宵交易時段（Overnight Trading）的流通量極低，僅佔Interactive Brokers總成交量的2%。",
    "盤前盤後（Extended Hours）的成交量約佔總成交量的10%。",
    "通宵交易時段只接受限價單（Limit Order），不接受市價單（Market Order）。"
  ],
  "takeaways": [
    "投資者現在可以在香港的辦公時間交易美股，無需熬夜。",
    "進行非正常時段交易時，必須注意流通性低、差價闊和滑價的風險。",
    "利用通宵交易時段執行預設的目標價訂單是較佳的應用方式，而非根據該時段的價格波動進行分析決策。",
    "由於流通量低，應避免在非正常時段進行大額交易。"
  ],
  "overall_summary": "這部影片詳細介紹了新興的美股24小時交易機制。它解釋了這是通過傳統交易所之外的ECN平台（如Blue 
Ocean）實現的，讓亞洲投資者可以在白天交易美股。影片提供了在Interactive 
Brokers平台上的詳細操作演示，並深入分析了24小時交易的利弊。其主要優點是便利性，但缺點是流通量低和交易成本可能增加（
差價擴大）。講者建議，投資者應理解真正的價格發現主要發生在正常交易時段，因此通宵交易更適合用來執行預先設定好的目標
價訂單，而不是用來判斷市場走勢。"
}

In [ ]:
final_result = Analysis.model_validate_json(final_result)

In [7]:
print(final_result)

Analysis(
    title='美股24小時交易：如何操作、優缺點及應用策略',
    chapters=[
        Chapter(
            header='什麼是美股24小時交易？',
            key_points=[
                '美股可透過電子通訊網絡（ECN）平台實現24小時交易。',
                '除了傳統交易所，還有ARCA、EDGX、Blue Ocean等ECN平台提供延長交易時段。',
                'ECN平台與傳統交易所交易的股票完全相同，只是交易渠道不同。'
            ],
            summary='影片介紹了美股現在可以24小時交易，解決了亞洲投資者需要熬夜的問題。這主要是通過傳統交易所（如紐
約證券交易所、納斯達克）之外的電子通訊網絡（ECN）平台實現的。這些ECN平台，如ARCA、EDGX和Blue 
Ocean，提供了盤前、盤後甚至通宵的交易時段，讓交易時間得以延長。這些平台與傳統交易所提供的股票是完全相同的，只是交易
渠道和時間不同。'
        ),
        Chapter(
            header='24小時美股交易實況操作',
            key_points=[
                '使用IB平台時，需在「Time In Force (TIF)」中選擇「Overnight Trading」。',
                '影片以買入特斯拉股票為例進行操作示範。',
                '下單前務必覆核訂單總金額，防止出錯。',
                '通宵交易時段只接受限價單（Limit Order）。'
            ],
            summary='影片以Interactive Brokers (IB) 
的交易平台為例，逐步示範如何在香港的日間時段（對應美國的夜間）買賣美股。操作關鍵在於下單時，在「Time In Force 
(TIF)」選項中選擇「Overnight 
Trading」。影片展示了如何為特斯拉（TSLA）股票下達一個通宵交易的買單，並強調了檢查訂單總金額（Amount）的重要性，以避
免下錯單。此外，通宵交易目前只接受限價單（Limit Order）。'
        ),
        Chapter(
            header='24小時交易的優點與缺點',
            key_points=[
                '優點：方便，可在亞洲時區對市場新聞作出即時反應。',
                '缺點：流通量低，買賣差價闊。',
                '缺點：訂單稀疏，大額交易容易造成滑價（Trade Slippage）。'
            ],
            summary='24小時交易的最大好處是方便，尤其對於亞洲投資者，可以在白天對突發事件作出反應，無需熬夜。然而，
缺點也很明顯，主要是在非正常交易時段（特別是通宵時段），市場流通量低，導致掛盤稀疏、買賣差價擴大，大額交易容易影響
價格並造成滑價（Slippage）風險。'
        ),
        Chapter(
            header='如何應用24小時交易策略',
            key_points=[
                '真正的價格發現（Price Discovery）發生在正常交易時段。',
                '不應根據通宵時段的價格波動來判斷市場趨勢或做交易決策。',
                '可利用通宵交易時段執行預先設定好的目標價位訂單。'
            ],
            summary='講者分享了他的個人策略。他認為真正的價格發現（Price 
Discovery）發生在交易量最大的正常交易時段（美國白天）。因此，他不會根據通宵時段的價格波動來做新的交易決策。相反，他
會利用這個時段來執行預先設定好的目標價位訂單，例如當股價達到某個預設的賣出價時，就可以掛單賣出。他用街市買魚作比喻
，日間的價格才反映真實的供需，深夜的零星交易價格不具代表性。'
        )
    ],
    key_facts=[
        '美股可透過ECN平台實現24小時交易。',
        'Blue Ocean ATS平台提供美國東岸時間晚上8點至凌晨4點的交易，對應香港夏令時間早上8點至下午4點。',
        '通宵交易時段（Overnight Trading）的流通量極低，僅佔Interactive Brokers總成交量的2%。',
        '盤前盤後（Extended Hours）的成交量約佔總成交量的10%。',
        '通宵交易時段只接受限價單（Limit Order），不接受市價單（Market Order）。'
    ],
    takeaways=[
        '投資者現在可以在香港的辦公時間交易美股，無需熬夜。',
        '進行非正常時段交易時，必須注意流通性低、差價闊和滑價的風險。',
        '利用通宵交易時段執行預設的目標價訂單是較佳的應用方式，而非根據該時段的價格波動進行分析決策。',
        '由於流通量低，應避免在非正常時段進行大額交易。'
    ],
    overall_summary='這部影片詳細介紹了新興的美股24小時交易機制。它解釋了這是通過傳統交易所之外的ECN平台（如Blue 
Ocean）實現的，讓亞洲投資者可以在白天交易美股。影片提供了在Interactive 
Brokers平台上的詳細操作演示，並深入分析了24小時交易的利弊。其主要優點是便利性，但缺點是流通量低和交易成本可能增加（
差價擴大）。講者建議，投資者應理解真正的價格發現主要發生在正常交易時段，因此通宵交易更適合用來執行預先設定好的目標
價訂單，而不是用來判斷市場走勢。'
)

In [ ]:
from youtube_summarizer.youtube_loader import youtube_loader

youtube_loader("https://www.youtube.com/watch?v=S07XJKq-H6Q")

In [ ]:
import os

from dotenv import load_dotenv
from google.genai import Client, types
from pydantic import BaseModel, Field
from rich import print

load_dotenv()


class Chapter(BaseModel):
    header: str = Field(description="A descriptive title for the chapter")
    key_points: list[str] = Field(description="Important takeaways and insights from this chapter")
    summary: str = Field(description="A comprehensive summary of the chapter content")


class Analysis(BaseModel):
    title: str = Field(description="The main title or topic of the video content")
    chapters: list[Chapter] = Field(description="Structured breakdown of content into logical chapters")
    key_facts: list[str] = Field(description="Important facts, statistics, or data points mentioned")
    takeaways: list[str] = Field(description="Key insights and actionable takeaways for the audience")
    overall_summary: str = Field(description="A comprehensive summary synthesizing all chapters, facts, and themes")


client = Client(api_key=os.getenv("GEMINI_API_KEY"))

response = client.models.generate_content(
    model="models/gemini-2.5-pro",
    contents=types.Content(
        parts=[types.Part(text=caption)],
    ),
    config=types.GenerateContentConfig(
        temperature=0,
        response_mime_type="application/json",
        response_schema=Analysis,
        thinking_config=types.ThinkingConfig(thinking_budget=2048),
    ),
)

print(response.text)

{
  "title": "An Investment Analysis of Intel (INTC)",
  "chapters": [
    {
      "header": "Introduction: The Next Big Stock Opportunity",
      "key_points": [
        "The video identifies Intel (INTC) as a stock with a textbook setup for a breakout, despite being down 
60%.",
        "The analysis is supported by institutional buying data from Darkpool trading, which has been consistently 
bullish on Intel.",
        "A free detailed workbook is offered to viewers for a deeper understanding of the company and Wall Street 
trading rules."
      ],
      "summary": "The speaker introduces Intel as a significant investment opportunity, comparing its potential to 
past successes like Palantir. He highlights that institutional investors ('the big boys') have been actively buying
the stock, as indicated by Darkpool data. The video promises to cover five key bullish reasons and a specific entry
point for purchasing the stock."
    },
    {
      "header": "The Five Bullish Reasons for Intel",
      "key_points": [
        "The US government is providing Intel with approximately $7.8 billion in grants to build domestic chip 
foundries, making it a critical piece of US infrastructure.",
        "New leadership is targeting $10 billion in cost savings and launching more competitive chips that are 
allegedly 50% cheaper to run than Nvidia's.",
        "The rise of AI is expected to drive a PC refresh cycle, as users will need more powerful machines for 
local computing, benefiting Intel.",
        "Intel is investing tens of billions to build a foundry business to rival TSMC and capture a share of the 
$80 billion third-party foundry market.",
        "Based on its current valuation, some analysts project a potential 3-5x return in the next few years if the
company successfully executes its turnaround plan."
      ],
      "summary": "This chapter details the five core fundamental arguments for being bullish on Intel. These 
include significant government support to onshore chip manufacturing, aggressive cost-cutting measures under new 
leadership, the growing demand for AI-capable PCs, the strategic expansion into the third-party foundry business to
compete globally, and the stock's attractive valuation which suggests a high potential for future returns."
    },
    {
      "header": "Technical Analysis and Entry Strategy",
      "key_points": [
        "The stock has been in a sideways consolidation phase for about a year, suggesting that most sellers have 
already exited their positions.",
        "Recent trading volume has been historically high, indicating a 'clean slate' is being created for a 
potential upward move.",
        "A key resistance level is identified at $25, where market makers and institutions are likely to sell.",
        "The recommended entry point is not now, but after the stock breaks out above the recent high of 
approximately $27.50 on significant volume."
      ],
      "summary": "The speaker shifts from fundamentals to technical analysis, examining Intel's stock chart. He 
notes a year-long consolidation period, which he views as a positive sign as it shakes out weak hands. He points 
out a significant resistance level at $25. The core message is to avoid buying prematurely and instead wait for a 
confirmed breakout above the $27.50 level, supported by high trading volume, as this would signal strong buying 
momentum and a higher probability of a sustained upward trend."
    },
    {
      "header": "Conclusion and Risk Management",
      "key_points": [
        "Investors should not blindly buy a stock based on a YouTube video or FOMO (Fear Of Missing Out).",
        "It is crucial to have a personal risk management strategy, including setting stop-losses.",
        "The provided workbook can help investors develop their own strategy and understand the rules for buying 
such stocks."
      ],
      "summary": "The video concludes with a strong caution against impulsive buying. The speaker emphasizes t